Simple example of eXirt's use

Run all bullets in the sequence below!



Generate files?


## Install dependences

In [23]:
!pip install catsim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
!pip install openml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import openml
import pandas as pd
import statistics
import numpy as np
import random
import copy


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from google.colab import files

## Install eXirt 

In [26]:
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_MLtIRT.py
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_analysis.py

--2023-02-24 13:23:26--  https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_MLtIRT.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7453 (7.3K) [text/plain]
Saving to: ‘decodIRT_MLtIRT.py.1’

decodIRT_MLtIRT.py. 100%[===================>]   7.28K  --.-KB/s    in 0s      

2023-02-24 13:23:26 (60.1 MB/s) - ‘decodIRT_MLtIRT.py.1’ saved [7453/7453]

--2023-02-24 13:23:26--  https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_analysis.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent,

In [27]:
!pip install eXirt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from pyexirt import eXirt as explainer

## Load data and Pre-process

In [18]:
def normalize(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        if(len(df_norm[column].unique()) > 1): #fix NaN generation
          df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        else:
          df_norm[column] = 0
    return df_norm

In [19]:
#Select dataset name by openml link https://www.openml.org/search?sort=date
dataset_name = "pc1"

In [20]:
#load dataset by OpenML

dataset = openml.datasets.get_dataset(dataset_name)
X, Y, categorical_indicator, attribute_names = dataset.get_data(
                  dataset_format="dataframe", target=dataset.default_target_attribute)

print(dataset)
#Preprocess Y and X numerics

if (Y.dtype != 'numeric'):
  Y = Y.astype(int)

for i,c in enumerate(X.columns):
  if (X[c].dtype != 'float64'):
    X = X.astype(float)

#Normalization
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y) # 70% training and 30% test



OpenML Dataset
Name..........: pc1
Version.......: 1
Format........: ARFF
Upload Date...: 2014-10-06 23:57:45
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/53951/pc1.arff
OpenML URL....: https://www.openml.org/d/1068
# of features.: 22
# of instances: 1109


## Creation and prediction model

In [21]:
model = RandomForestClassifier(200)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

## Global explanation rank

In [29]:
explainer = explainer.eXirt()

In [30]:
global_explanation_attributes, global_explanation_attributes_scores = explainer.explainRankByEXirt(model, X_train, X_test, y_train, y_test,dataset_name)

In [31]:
global_explanation_attributes

['locCodeAndComment',
 'lOComment',
 'lOBlank',
 'N',
 'uniq_Op',
 'uniq_Opnd',
 'loc',
 'total_Op',
 'lOCode',
 'T',
 'B',
 'E',
 'I',
 'D',
 'L',
 'V',
 'iv(G)',
 'ev(g)',
 'v(g)',
 'total_Opnd',
 'branchCount']

In [32]:
global_explanation_attributes_scores

,Final Score
locCodeAndComment,-113.913506
lOComment,-113.776775
lOBlank,-113.670619
N,-113.549347
uniq_Op,-113.534192
uniq_Opnd,-113.519036
loc,-113.488725
total_Op,-113.488725
lOCode,-113.488725
T,-113.488725
